# Match BGBM Collectors to Wikidata Items Using *Nearest Neighbour*, Just Name Comparison

See also [match_names_BGBM-dwcagent-parsed-eventDate_vs_WikiData_k-nearest.ipynb](./match_names_BGBM-dwcagent-parsed-eventDate_vs_WikiData_k-nearest.ipynb) that includes the `eventDate` of sampling to get a life time reference for deciding name matches to WikiData.

- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

For the output of DarwinCore agent attribution, reconsidere `displayOrder` that it represents rather the data quality first and foremost, *not* the very name match.

Technical Notes — Review Code perhaps:
- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Use Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb) to generate matching data of botanists.

Now load the data and make them unique …

In [38]:
import pandas as pd
import pprint, time, os

wikidata = pd.read_csv(
    # "data/wikidata_persons_botanists_20231030_1539.csv", # inverse match: [particle +] family, given
    "data/wikidata_persons_botanists_20231116.csv",        # match: given [+ particle] + family[+ , suffix]
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }    
)
pprint.pprint(wikidata.columns)
display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link',
       'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='object')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Friedrich August Marschall von,F. A. M. v.,F. A. M. v. Bieberstein,Friedrich August Marschall von Bieberstein,NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Hans Hermann,H. H.,H. H. Behr,Hans Hermann Behr,NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818,1904,<NA>,<NA>,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Jacob Christian,J. C.,J. C. Schäffer,Jacob Christian Schäffer,NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718,1790,<NA>,<NA>,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Johann Friedrich,J. F.,J. F. Klotzsch,Johann Friedrich Klotzsch,NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805,1860,<NA>,<NA>,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Franz Anton,F. A.,F. A. Menge,Franz Anton Menge,NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808,1880,<NA>,<NA>,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [39]:
# Create data frame with unique canonical strings 
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

display(wd_matchtest)
display(wd_matchtest_fullnames)

# colls = list(wikidata.columns)

,canonical_string,item
,,count
0,('W.') S. W. Wong,1
1,(A. A.) G. L. Monnier,1
2,(A.) F. Valet,1
3,(A.) H. (S.) Stenar,1
4,(A.) T. Wegelin,1
...,...,...
69099,Э. Э. Керн,1
69100,Ю. К. Шель,1
69101,Ю. П. Нюкша,1


,canonical_string_fullname,item
,,count
0,('Wilson') Sze Wing Wong,1
1,(Alexandre Alexis) George Le Monnier,1
2,(Antonius) Theodoor Wegelin,1
3,(August) Friedrich Valet,1
4,(Axel) Helge (Svensson) Stenar,1
...,...,...
71394,Эдуард Эдуардович Керн,1
71395,Юлиан Карлович Шель,1
71396,Юлия Петровна Нюкша,1


### Load Collectors Data Set

Data sources:

- option 1: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_dataset.ipynb)
- option 2: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

In [3]:
# atomized names were parsed already by ruby gem package: dwcagent —
# they can contain also the same name accross multiple rows — 
# it’s probably better for the matching to make the name rows unique later on

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_occurrenceIDs_20230524_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.
collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample
3,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano,E.",parsed:E. A. Cano,cleaned:E. A. Cano,1,https://herbarium.bgbm.org/object/B100699397
39972,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murashige & Aaiki,parsed:Murashige<SEP>Aaiki,cleaned:Murashige<SEP>Aaiki,1,https://herbarium.bgbm.org/object/B101149305
7,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn,A.",parsed:A. Aaronsohn,cleaned:A. Aaronsohn,3,https://je.jacq.org/JE00010154
27181,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M.Ait Lafki...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,3,https://herbarium.bgbm.org/object/B100217620
27185,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A., Ait Lafkih,M. & Griffith...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Lafk...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,2,https://herbarium.bgbm.org/object/B100326682
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16202,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Tamanyan,...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,4,https://herbarium.bgbm.org/object/B100559866
16162,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,1,https://herbarium.bgbm.org/object/B100575827
16154,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,7,https://herbarium.bgbm.org/object/B100810598
17787,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich,parsed:Gabrielian Գաբրիելյան<SEP>Габриелян<SEP...,cleaned:Gabrielian Գաբրիելյան<SEP>Габриелян<SE...,1,https://herbarium.bgbm.org/object/B100088191


#### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (1376 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample
2328,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Bally,P.R.O., Abdallah, Raffael & Reichstein, T.",parsed:P.R.O. Bally<SEP>Raffael Abdallah<SEP>T...,cleaned:P.R.O. Bally<SEP>Raffael Abdallah<SEP>...,1,https://herbarium.bgbm.org/object/B200125981
27272,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,Kadir Bin Abdul,parsed:Kadir Bin Abdul,cleaned:Kadir Bin Abdul,1,https://herbarium.bgbm.org/object/B100184021
21225,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,Guilherme de Abreu (no. 103),parsed:Guilherme de Abreu,cleaned:Guilherme de Abreu,1,http://id.snsb.info/snsb/collection/22086/3086...
18644,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"García Adá, Luceño, Rico,E., Romero,T. & Varga...",parsed:García Adá<SEP>Rico Luceño<SEP>E. Romer...,cleaned:García Adá<SEP>Rico Luceño<SEP>E. Rome...,1,https://herbarium.bgbm.org/object/B100296455
417,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,"Ahagen, Schiers,C. & al.",parsed:Schiers Ahagen<SEP>C.<SEP>Al,cleaned:Schiers Ahagen<SEP>C.<SEP>Al,1,https://herbarium.bgbm.org/object/B100194646
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38926,Мирзоева,Mirzoeva,NaN,NaN,NaN,NaN,NaN,NaN,Mirzoeva [Мирзоева] & Gambarian & Pogosian,parsed:Mirzoeva Мирзоева<SEP>Gambarian<SEP>Pog...,cleaned:Mirzoeva Мирзоева<SEP>Gambarian<SEP>Po...,1,https://herbarium.bgbm.org/object/B100355321
43846,Попов,Popov,NaN,NaN,NaN,NaN,NaN,NaN,Popov [Попов],parsed:Popov Попов,cleaned:Popov Попов,3,https://dr.jacq.org/DR049416
43847,Попов,Popov,NaN,NaN,NaN,NaN,NaN,NaN,Popov [Попов] & Vvedensky,parsed:Popov Попов<SEP>Vvedensky,cleaned:Popov Попов<SEP>Vvedensky,4,https://herbarium.bgbm.org/object/B101149435
17787,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich,parsed:Gabrielian Գաբրիելյան<SEP>Габриелян<SEP...,cleaned:Gabrielian Գաբրիելյան<SEP>Габриелян<SE...,1,https://herbarium.bgbm.org/object/B100088191


In [5]:
# check the parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))


----------------------------------------
show names with **particle** found 742 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
21225,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN
365,Aghababyan,M.,NaN,von,NaN,NaN,NaN,NaN
4212,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
61135,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
16932,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 15 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17455,August,Friedrich,II.,NaN,NaN,NaN,NaN,NaN
59171,Dogma,I.J.,Jr.,NaN,NaN,NaN,NaN,NaN
17185,Forsyth,W.,jr.,NaN,NaN,NaN,NaN,NaN
808,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN
26390,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17287,Sennen,NaN,NaN,NaN,NaN,NaN,Fr,NaN


Compile and compose `canonical_string…` of the collector data that we will later match the WikiData names with:

In [7]:
# combine parts of names similar to WikiData's given name labels
collectors['canonical_string_collector_parsed'] = collectors[['given', 'particle', 'family', 'suffix']]\
    .fillna('')\
    .apply(
        lambda this_df: "{given}{particle}{family}{suffix}".format(
            given=this_df["given"],
            particle=" " + this_df["particle"] if this_df["particle"] else '', 
            family=" " + this_df["family"] if this_df["family"] else '', 
            suffix=", " + this_df["suffix"] if this_df["suffix"] else ''
        ), axis="columns"
    )

criterion = collectors["particle"].str.contains("\w+ \w+", na=False)

# display(collectors['canonical_string_collector_parsed'][criterion].head())
collectors[['canonical_string_collector_parsed', 'particle']][criterion].drop_duplicates().head(10)


,canonical_string_collector_parsed,particle
4212,M.L. Reyna de Aguilar,Reyna de
8,H. Abbas al Ani,Abbas al
49324,J. Francisco del Aquila,Francisco del
34600,A.L.M. Marcailhou d' Aymeric,Marcailhou d'
47421,P.R.O. Ritchie in Bally,Ritchie in
42379,H. Perrier de la Bathie,Perrier de la
41744,A.M.F. Palisot de Beauvois,Palisot de
41745,A.M.F.J. Palisot de Beauvois,Palisot de
60691,D. Van der Ben,Van der
41756,F.A. Marschall von Bieberstein,Marschall von


In [8]:
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
16202,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Tamanyan,..."
16162,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian..."
16154,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian..."
17787,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian Գաբրիելյան,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich
59137,Թամանյան,Tamanian,NaN,NaN,NaN,NaN,NaN,NaN,Tamanian Թամանյան,Tamanian [Թամանյան; Таманян]


In [9]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]) # custom function, to get the first entry
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample
0,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,1,https://herbarium.bgbm.org/object/B101149305
1,Abbot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abbot,Abbot,2,https://herbarium.bgbm.org/object/B100159967
2,Abdelkrim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abdelkrim,"Abdelkrim & Wanke,S.",1,https://dr.jacq.org/DR052220
3,Abel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abel,Abel,1,https://herbarium.bgbm.org/object/B100384250
4,Abner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abner,Abner,1,https://herbarium.bgbm.org/object/B180021124
...,...,...,...,...,...,...,...,...,...,...,...,...
20948,Таманян,К.,NaN,NaN,NaN,NaN,NaN,NaN,К. Таманян,"Tamanian,K. [Թամանյան,Կ.; Таманян,К.]",16,https://herbarium.bgbm.org/object/B100190731
20949,Оганесиан,М.,NaN,NaN,NaN,NaN,NaN,NaN,М. Оганесиан,"Oganesian,M. [Ոգանեսիան,Մ.; Оганесиан,М.], Ter...",25,https://herbarium.bgbm.org/object/B100275643
20950,Androsov,Н.,NaN,NaN,NaN,NaN,NaN,NaN,Н. Androsov,"Androsov, Н. (no. 2404a)",1,http://id.snsb.info/snsb/collection/455833/556...
20951,Габриелян,Н.,NaN,NaN,NaN,NaN,NaN,NaN,Н. Габриелян,"Gabrieljan,N. [Գաբրիելյան,Ն.; Габриелян,Н.] & ...",1,https://herbarium.bgbm.org/object/B100190831


In [10]:
# show collectors with highest occurrenceID_collectors_count
collectors_unique.sort_values(by=['occurrenceID_collectors_count', 'family', 'given'], ascending=[False, True, True]).head(10)

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample
8554,Willing,E.,NaN,NaN,NaN,NaN,NaN,NaN,E. Willing,"Willing,E. & Eisenblätter,R.",189994,https://herbarium.bgbm.org/object/B100145955
18275,Willing,R.,NaN,NaN,NaN,NaN,NaN,NaN,R. Willing,"Willing,R. & Willing,R.",188088,https://herbarium.bgbm.org/object/B100074611
17105,Hein,P.,NaN,NaN,NaN,NaN,NaN,NaN,P. Hein,"Kilian,N., Hein,P. & Oberprieler,C.",9594,https://herbarium.bgbm.org/object/B100113011
16393,Kilian,N.,NaN,NaN,NaN,NaN,NaN,NaN,N. Kilian,"Hunger,S. & Kilian,N.",9198,https://herbarium.bgbm.org/object/B100003448
7057,Martius,C.F.P.,NaN,von,NaN,NaN,NaN,NaN,C.F.P. von Martius,"Martius, C.F.P. von",6097,http://id.snsb.info/snsb/collection/502732/623...
7603,Rodríguez,D.,NaN,NaN,NaN,NaN,NaN,NaN,D. Rodríguez,"Rodríguez,D., Monterrosa,J., Hernández,A. & Ma...",5206,https://herbarium.bgbm.org/object/B100038970
8483,Tempel,E.,NaN,NaN,NaN,NaN,NaN,NaN,E. Tempel,"Tempel,E.",4561,https://dr.jacq.org/DR073621
6393,Martius,C. F. P. von,NaN,NaN,NaN,NaN,NaN,NaN,C. F. P. von Martius,"Martius, C.F.P. von (no. Obs. 1490)",4545,http://id.snsb.info/snsb/collection/117775/176...
10706,Schimper,G.W.,NaN,NaN,NaN,NaN,NaN,NaN,G.W. Schimper,"Schimper, G.W. (no. 880)",4320,http://id.snsb.info/snsb/collection/108223/167...
11620,Haussknecht,H.K.,NaN,NaN,NaN,NaN,NaN,NaN,H.K. Haussknecht,"Haussknecht, H.K.",4288,http://id.snsb.info/snsb/collection/474055/583...


In [11]:
# Idea: Should we use data column suffixes to follow the data source after merging is done later?
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

### Set Up the Text Analysis

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 for deeper understanding.

The `ngrams` function is used as an analyzer in the text search later.

In [12]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text
     
    @param string: the text string to perform the ngram splitting on 
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD', r'', string)
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [13]:
# pip install --upgrade scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# nbrs_data = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf_vector_data) 
# tfidf_vector_data contains the vectorized wikidata names from the previous step


def getNearestNeighbour(query, this_vectorizer, this_nbrs_data):
    """Calculate the k-nearest distance for query data using package scikit-learn


    @param query: DataFrame the query data to vectorize and transform
    @param this_vectorizer: the vectorizer of TfidfVectorizer
    @param this_nbrs_data: the data of NearestNeighbors calculations
    @return: (distances, indices) distances and indices
    @rtype (int, int)
    """
    queryTFIDF_ = this_vectorizer.transform(query)
    distances, indices = this_nbrs_data.kneighbors(queryTFIDF_)
    return distances, indices


def calculateTFIDFmatchingOfData(query_data, match_data, n_neighbors=1):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with getNearestN

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param n_neighbors: Number of neighbors required for each sample by default for :meth:`kneighbors` queries (originally 5).

    @requires NearestNeighbors()
    @requires getNearestNeighbour()
    @requires ngrams()
    @requires TfidfVectorizer()
    @requires NearestNeighbors()

    @return: DataFrame a data frame of matches with columns 'namematch_source_data', 'namematch_resource_data', 'namematch_distance'
    """

    import time
    start = time.time()
    query_data = set(query_data)
    # convert list to set for better performance

    print('Vectorizing data. This may take a while...')
    # vectorize wikidata names
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
    tfidf_vector_data = vectorizer.fit_transform(match_data
        # wd_matchtest['canonical_string']
    )
    nbrs_data = NearestNeighbors(n_neighbors=n_neighbors, n_jobs=-1).fit(tfidf_vector_data)
    duration = time.time() - start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    print('Getting nearest neighbours of %s data with %s neighbor sample(s)...' % (len(query_data), n_neighbors))
    distances, indices = getNearestNeighbour(query_data, vectorizer, nbrs_data)
    duration = time.time() - start
    print('Completed after %s s' % duration)

    query_data = list(query_data)  # convert back to list

    print('Finding matches build new data frame ...')
    matches = []
    for i, j in enumerate(indices):
        temp = [query_data[i], match_data.values[j][0], round(distances[i][0], 2)]
        matches.append(temp)

    duration = time.time() - start
    print('Building matches done after %s s' % duration)
    matches = pd.DataFrame(
        matches,
        columns=['namematch_source_data', 'namematch_resource_data', 'namematch_distance']
    )

    print('Done')
    return matches


Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/.

In [15]:
# some example data
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[0]))

# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('\n(WikiData’s) canonical_string = (constructed) canonical_string_fullname:') 
    print("- {short_name} = {long_name}".format(
        short_name=wd_matchtest['canonical_string'].at[row],
        long_name=wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))


Show ngram examples:
- simple name: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
- data from collectors: [' Ab', 'Abb', 'bbo', 'bot', 'ot ']
- data from match-test: [' W ', 'W S', ' S ', 'S W', ' W ', 'W W', ' Wo', 'Won', 'ong', 'ng ']
- data from match-test (full name): [' Wi', 'Wil', 'ils', 'lso', 'son', 'on ', 'n S', ' Sz', 'Sze', 'ze ', 'e W', ' Wi', 'Win', 'ing', 'ng ', 'g W', ' Wo', 'Won', 'ong', 'ng ']

(WikiData’s) canonical_string = (constructed) canonical_string_fullname:
- ('W.') S. W. Wong = ('Wilson') Sze Wing Wong
- (A. A.) G. L. Monnier = (Alexandre Alexis) George Le Monnier
- (A.) F. Valet = (Antonius) Theodoor Wegelin
- (A.) H. (S.) Stenar = (August) Friedrich Valet
- (A.) T. Wegelin = (Axel) Helge (Svensson) Stenar


### Perform the Matching

Perform the nearest neighbour (NN) matches on the (BGBM) collector names and create a data frame with matches, and we try to distinguish abbreviated and full names in the source to better match source data and WikiData ... (can take 5 to 10 minutes)

Now convert a collection of raw documents to a matrix of TF-IDF features and set up the function that performs the matches...

In [16]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values
# collectors_names = set(collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values)
print("Calculate abbrevated names only …")
matches = calculateTFIDFmatchingOfData(collectors_names, wd_matchtest['canonical_string'], 5) 
        # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index(names=['old_index'])

display(matches)

Calculate abbrevated names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.411162853240967 s
Getting nearest neighbours of 20414 data with 5 neighbor sample(s)...
Completed after 147.36264157295227 s
Finding matches build new data frame ...
Building matches done after 147.56478118896484 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,20413,D. Alexander,D. Alexander,0.0
1,6459,B. Girerd,B. Girerd,0.0
2,6455,S. Brunies,S. Brunies,0.0
3,6448,М. Оганесиан,І. Г. Васількоў,0.0
4,17360,J. Roell,J. Roell,0.0
...,...,...,...,...
20409,5888,H. Gunckel,І. І. Гранітаў,1.0
20410,5884,Mühlig,І. І. Гранітаў,1.0
20411,17672,M. Wiljamsson,І. М. Сцепановіч,1.0
20412,5867,W.J. Howrigon,І. Г. Васількоў,1.0


In [17]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
print("Calculate full names only …")
matches_fullnames = calculateTFIDFmatchingOfData(collectors_fullnames, wd_matchtest_fullnames['canonical_string_fullname'], 5) 
         # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches_fullnames = matches_fullnames.sort_values(['namematch_distance'])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

display(matches_fullnames)


Calculate full names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.750764846801758 s
Getting nearest neighbours of 539 data with 5 neighbor sample(s)...
Completed after 11.872997999191284 s
Finding matches build new data frame ...
Building matches done after 11.878899097442627 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,363,André Schmitz,André Schmitz,0.0
1,136,Leopoldo Nicotra,Leopoldo Nicotra,0.0
2,455,Emanuel Weiss,Emanuel Weiss,0.0
3,141,Justus J.H. Schmidt,Justus J.H. Schmidt,0.0
4,143,Gustav Maass,Gustav Maass,0.0
...,...,...,...,...
534,313,Shaik Mokim,Алден Спрингер Крафтс,1.0
535,316,Abel el Khalek,Алден Спрингер Крафтс,1.0
536,319,Grupo Botánico Gallego,Іпаліт Іванавіч Гранітаў,1.0
537,287,Alois Wilfling,Іпаліт Іванавіч Гранітаў,1.0


### Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

In [18]:
# join matches data frame back to source collectors  dataframe 
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,1,https://herbarium.bgbm.org/object/B101149305,594,Aaiki,K. Saiki,0.86
1,Abbot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abbot,Abbot,2,https://herbarium.bgbm.org/object/B100159967,13612,Abbot,J. Abbot,0.37
2,Abdelkrim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abdelkrim,"Abdelkrim & Wanke,S.",1,https://dr.jacq.org/DR052220,19424,Abdelkrim,І. Г. Васількоў,1.00
3,Abel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abel,Abel,1,https://herbarium.bgbm.org/object/B100384250,10860,Abel,C. Abel,0.47
4,Abner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abner,Abner,1,https://herbarium.bgbm.org/object/B180021124,20063,Abner,І. І. Гранітаў,1.00


In [19]:
# append full name matches
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,Orbigny,ACVDd',NaN,NaN,NaN,NaN,NaN,NaN,ACVDd' Orbigny,"Orbigny,A.C.V.D.d'",10,https://herbarium.bgbm.org/object/B200097228,195,ACVDd' Orbigny,Charles d'Orbigny,1.00
1,Sauze,Abbé,NaN,NaN,NaN,NaN,NaN,NaN,Abbé Sauze,"Sauze,Abbé,J.",1,https://herbarium.bgbm.org/object/B100263049,167,Abbé Sauze,Алден Спрингер Крафтс,1.00
2,Hassan,Abdisalam Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalam Sheikh Hassan,"Friis,I., Vollesen,K. & Abdisalam Sheikh Hassan",7,https://herbarium.bgbm.org/object/B100003700,331,Abdisalam Sheikh Hassan,Abdisalam S. Hassan,0.69
3,Hassan,Abdisalem Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalem Sheikh Hassan,"Friis,I, Vollesen,K., & Abdisalem Sheikh Hassan",2,https://herbarium.bgbm.org/object/B100003663,309,Abdisalem Sheikh Hassan,Abdisalam S. Hassan,0.85
4,Khalek,Abel,NaN,el,NaN,NaN,NaN,NaN,Abel el Khalek,Abel el Khalek,1,https://herbarium.bgbm.org/object/B100763849,316,Abel el Khalek,Алден Спрингер Крафтс,1.00


In [20]:
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_distance
4106,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",3,https://je.jacq.org/JE00010154,951,A. Aaronsohn,A. Aaronsohn,0.0
18158,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,S. Abedin,"Abedin,S. & Hussein,A.",14,https://herbarium.bgbm.org/object/B100046632,6974,S. Abedin,S. Abedin,0.0
4108,Acebey,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Acebey,"Acebey, A. (no. 563)",6,http://id.snsb.info/snsb/collection/719469/786...,11602,A. Acebey,A. Acebey,0.0
14749,Ackermann,M.,NaN,NaN,NaN,NaN,NaN,NaN,M. Ackermann,"Ackermann, M. (no. 647)",34,http://id.snsb.info/snsb/collection/444683/544...,13442,M. Ackermann,M. Ackermann,0.0
14084,Acosta,L.,NaN,NaN,NaN,NaN,NaN,NaN,L. Acosta,"González,A.F.R., Ramírez,V. [Victor], Acosta,L...",538,https://herbarium.bgbm.org/object/B100542413,20264,L. Acosta,L. Acosta,0.0


In [21]:
# criterion = collectors_all_matches['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with namematch distances from 0.0 to 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
# collectors_all_matches[criterion]

In [22]:
# Save the plain name matching results only ...

do_custom_data_aggregation=False
if do_custom_data_aggregation:
    if not os.path.exists('data'):
        print("Make data directory for saving …")
        os.makedirs('data')
    
    # Set some global varialbes
    # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
    this_timestamp_for_data=20231116
    
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_plain-names_%s.csv' % (
        this_timestamp_for_data
    )
    
    collectors_all_matches.to_csv(this_output_file)
    
    print("Wrote plain name matches of collector names into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

### Merge and Aggregate Matched Data and WikiData’s

Review (TODO)
- merge abbreviated and full name data properly, distinguish abbrevited match and full name match
- refactor `collectors_matches` or `collectors_matches_g1` aso. to `collectors_all_matches`
- refactor `collectors` to `collectors_unique`
- refactor `matches`to `matches_abbr` or distinguish `matches_fullname`

Now
1. merge now the matching data and the wiki data’s on the conaonical string name
2. later aggregate fine tuned, checking if multiple same (canonical string) names relate to multiple different persons (we use wd-items (the Q1233242 thing), and wd-item-labels to aggregate on) … aso.
3. save those data tables

In [23]:
# merge now the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [24]:
print("Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
print("Interpretation: most of the matches seem correct (=Carl Georg Theodor) also with higher distances but we cannot be sure in mid ranges …")
print("Interestingly with an …yi: Kotschyi, C.G.T. (0.92) was matched to Carl Georg Theodor (=correct, and higher distance) …")
print("      … but Kotschy, C.G.T. (0.76) was calculated to the other person Carl Friedrich (=incorrect, lower distance … :-/)")

criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].str.contains('Kotschy')
collectors_matches_g1_merged_wikidata[criterion].get([
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    # 'canonical_string_fullname', 
    'itemLabel', 'wikidata_link'
])

Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)
Interpretation: most of the matches seem correct (=Carl Georg Theodor) also with higher distances but we cannot be sure in mid ranges …
Interestingly with an …yi: Kotschyi, C.G.T. (0.92) was matched to Carl Georg Theodor (=correct, and higher distance) …
      … but Kotschy, C.G.T. (0.76) was calculated to the other person Carl Friedrich (=incorrect, lower distance … :-/)


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,itemLabel,wikidata_link
6843,2,https://dr.jacq.org/DR049432,Kotschy,T. Kotschy,0.39,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
6844,1,http://id.snsb.info/snsb/collection/22980/3175...,C.G. Kotschy,T. Kotschy,0.75,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
6845,2494,http://id.snsb.info/snsb/collection/108230/167...,C.G.T. Kotschy,T. Kotschy,0.75,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
6846,1,https://herbarium.bgbm.org/object/B100160086,C.G.T. Kotschyi,T. Kotschy,0.89,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
6847,37,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,T. Kotschy,0.75,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
6848,310,http://id.snsb.info/snsb/collection/117808/176...,T. Kotschy,T. Kotschy,0.00,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
6850,5,https://herbarium.bgbm.org/object/B100526350,Th Kotschy,T. Kotschy,0.70,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
22200,2,https://je.jacq.org/JE00022436,Carl Georg Theodor Kotschy,Theodor Kotschy,0.59,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
22395,1,https://herbarium.bgbm.org/object/B101113772,Karl Georg Th Kotschy,Carl Friedrich Kotschy,0.93,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842


Aggregate data to get atomized listings of multiple resource name matches joining by “|” aso.

In [25]:
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    print('Group data by canonical names (abbreviated and full name): '
          '1st multiple related WD items (e.g. Q1232456), 2nd multiple WD item labels')
    for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
        print('Run %s: Group by wiki data’s %s, and aggregate/join item(s) '
              'by “…|…”, add new column items_joined ...' % (i + 1, wd_matching_column))
        wdata_joined_items = wikidata.groupby([wd_matching_column])['item'].apply('|'.join).reset_index()
        wdata_joined_items.rename(columns={'item': 'items_joined'}, inplace=True)
    
        # print("Done. Show examples of items having multiple matching data «|» … ")
        # criterion = wdata_joined_items['items'].map(lambda x: '|' in x)
        # wdata_joined_items[criterion].head()
    
        print('Run %s:   Merge all based on namematch_resource_data, add item(s) data ...' % (i + 1))
        collectors_matches_g2 = pd.merge(
            collectors_matches_g1_merged_wikidata, wdata_joined_items,
            left_on='namematch_resource_data', right_on=wd_matching_column
            , suffixes=('__wikidata_merge', '__grp_by_items')
            # append to left-data, right-data only when identical column names occur
        )
    
        # link wikidata items with canonical name string (pipe separated if more than one)
        print('Run %s:   Group by wiki data’s %s, and aggregate/join itemLabel(s) '
              'by “…|…”, add new column item_labels_joined ...' % (i + 1, wd_matching_column))
        wdata_joined_itemlabels = wikidata.groupby([wd_matching_column])['itemLabel'].apply('|'.join).reset_index()
        wdata_joined_itemlabels.rename(columns={'itemLabel': 'item_labels_joined'}, inplace=True)
    
        print('Run %s:   Merge all based on namematch_resource_data, add itemLabel(s) data ...' % (i + 1))
        collectors_matches_g3 = pd.merge(
            collectors_matches_g2, wdata_joined_itemlabels,
            left_on='namematch_resource_data', right_on=wd_matching_column
            , suffixes=('__grp_by_items', '__grp_by_item_labels')
            # append to left-data, right-data only when identical column names occur
        )
    
        print('Run %s:   Build data frame “collectors_matches_group” ...' % (i + 1))
        collectors_matches_group = collectors_matches_g3 \
            if i == 0 \
            else pd.concat([collectors_matches_group, collectors_matches_g3], ignore_index = True)
        
    print('Done')

In [26]:
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    print("Show examples of item_labels_joined having multiple matching data «|» … ")
    criterion = collectors_matches_group['item_labels_joined'].map(lambda x: '|' in x)
    
    collectors_matches_group[criterion].get([ # empty 
        # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
        # 'canonical_string_fullname', 
        'item_labels_joined', 'items_joined'
    ], default="...get: Are data empty or it has probably a wrong named column?")


In [27]:
# check what columns we have and what we would keep for further analysis and what to drop
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    pprint.pprint(collectors_matches_group.columns)
# from merge: _x would means normally from left column, _y means from right column
# in BASH fold text long lines; echo "${text}" | fold --spaces | sed 's@^@#  @'

Prepare data to save later on …

In [28]:
# Remove superfluous columns TODO check WARNING: A value is trying to be set on a copy of a slice from a DataFrame
# TODO check duplicates
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    collectors_matches_group_simplified = collectors_matches_group.get(
        ['family', 'given', 'canonical_string_collector_parsed', 
          'namematch_source_data', # redundant: 'namematch_source_data' == 'canonical_string_collector_parsed'
         'namematch_resource_data', 'namematch_distance', 
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye',
        'items_joined', 'canonical_string', 'canonical_string_fullname', 'surname', 'initials', 'item_labels_joined'
        ], default="...get: Are data empty or it has probably a wrong named column?"
    )
    # collectors_matches_group = collectors_matches_g3
    collectors_matches_group_simplified.sort_values(
        by=['namematch_distance', 'canonical_string_collector_parsed']
        , inplace=True
    )
    collectors_matches_group_simplified.drop_duplicates(inplace=True)
    collectors_matches_group_simplified.head()

In [29]:
# old file bgbm_collectors_matches_wikidata_items_group_concat_%s.csv
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_wditems_group_concat_wdlabels-joined_%s.csv' % (
        this_timestamp_for_data
    )
    
    collectors_matches_group.to_csv(this_output_file)
    
    print("Wrote groups of collectors matches into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

### Merge Data to Individual WikiData Items

For this, merge by namematch_resource_data and focus to get individual WikiData items.

In [30]:
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    print('Merge simply namematch_resource_data to Wiki data for abbreviated and full names... ')
    for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
    
        # join wikidata items to avh collectors matches
        #   avh_matches = pd.merge(avh, matches, left_on='label', right_on='name')
        #   avh_matches_t1 = pd.merge(avh_matches, wikidata, left_on='matched_name', right_on='canonical_string')
        # link counts of wikidata items with same canonical name string
        #   avh_matches_t2 = pd.merge(avh_matches_t1, wd_test, left_on="matched_name", right_on="canonical_string")
        #   avh_matches_t2.rename(columns = {list(avh_matches_t2.columns)[-1]: 'dup_count'}, inplace=True)
        
        print('Run %s:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...' % (i + 1))
        collectors_matches_wd1 = pd.merge(
            collectors_all_matches, wikidata,
            left_on='namematch_resource_data', right_on=wd_matching_column,
            suffixes=('__coll_all_matches', '__wd')
            # append to left-data, right-data only when identical column names occur
        )
    
        print('Run %s:   Build data frame “collectors_matches_with_wdata” ...' % (i + 1))
        collectors_matches_with_wdata = collectors_matches_wd1 \
            if i == 0 \
            else pd.concat([collectors_matches_with_wdata, collectors_matches_wd1], ignore_index=True)
    
    print('Done')


In [31]:
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    pprint.pprint(collectors_matches_with_wdata.columns)
    # echo "${text}" | fold --spaces | sed 's@^@#  @'

    collectors_matches_with_wdata.drop_duplicates(inplace=True)
    display(collectors_matches_with_wdata)

Save all columns for further analysis

In [32]:
# old bgbm_collectors_matches_wikidata-botanists_all-columns_%s.csv
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv' % (
        this_timestamp_for_data
    )
    
    collectors_matches_with_wdata.sort_values(
        by=['namematch_distance', 'canonical_string_collector_parsed']
        , inplace=True
    )
    collectors_matches_with_wdata.to_csv(
        this_output_file, index=False # drop index column
    )
    
    print("Wrote isolated WikiData items of collector matches into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

In [33]:
# TODO meaningful?
# remove redundant (duplicate (?or empty?)) columns that in any kind are duplicate data (i.e. that we usually do not need)
# do it by transposing it (https://www.statology.org/pandas-drop-duplicate-columns/)
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    compact_df_tmp=collectors_matches_with_wdata.transpose().drop_duplicates().transpose()
    compact_df_tmp.sort_values(
        by=['namematch_distance', 'canonical_string_collector_parsed']
        , inplace=True
    )
    
    # old bgbm_collectors_matches_wikidata-botanists_all-columns-made-unique_%s.csv
    # results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns-compact_%s.csv' % (
        # "20230823"
        time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
    )
    
    compact_df_tmp.to_csv(
        this_output_file, index=False # drop index column
    )
    
    print("Wrote isolated WikiData items (unique columns) of collector matches into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

In [34]:
# refactor collectors_eventDate_mean
# refactor collectors_eventDate_min
# - refactor yob_is_lt_eventDate_min
# refactor collectors_eventDate_max
# - refactr yod_is_gt_eventDate_max
# refactor custom_score_lifetime            → custom_score_lifetime_data
# refactor custom_score_lifetime_annotation → custom_score_lifetime_data_annotation
# refactor namematch_similarity             → namematch_distance
# refactor namematch_similarity_annotation  → namematch_distance_annotation
collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
     'yob', 'yod', 'wyb'
    ]
]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossim.sort_values(
    by=['namematch_distance', 'family', 'given'], 
    ascending=[True, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossim.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_distance", 
    "source_data", 
    "itemLabel", 
    "item",
    'yob', 'yod'
]).copy()

dwcagent_attr_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
dwcagent_attr_output['namematch_distance_annotation'] = dwcagent_attr_output['namematch_distance'].astype(str).str.replace(r'(.+)', '\\1 (k-means distance)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_distance_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["custom_score_lifetime_data"] = 0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_data_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & pd.notnull(dwcagent_attr_output["yod"]),
    "custom_score_lifetime_data"
] = 1
# True cases but <NA> missing values
dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & dwcagent_attr_output["yod"].isnull(),
    "custom_score_lifetime_data"
] = 0.5
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & pd.notnull(dwcagent_attr_output["yod"]),
    "custom_score_lifetime_data"
] = 0.5
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & dwcagent_attr_output["yod"].isnull(),
    "custom_score_lifetime_data"
] = 0


# annotations True cases
dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & pd.notnull(dwcagent_attr_output["yod"]), 
    "custom_score_lifetime_data_annotation"
] = "life time known"

# annotations True cases but <NA> missing values
dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & dwcagent_attr_output["yod"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "year of death is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & pd.notnull(dwcagent_attr_output["yod"]), 
    "custom_score_lifetime_data_annotation"
] = "year of birth is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & dwcagent_attr_output["yod"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "unknown life time"


dwcagent_attr_output["custom_score_multiple_names"]=0 # 0 shall mean: we don’t know yet for real
dwcagent_attr_output.loc[
    (dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

namematch_distance_max=dwcagent_attr_output['namematch_distance'].max()
dwcagent_attr_output['custom_score_overall'] = (
    # reconsider/transform distance (0 … xx, range larger than 1) to similarity (1 … 0, range of 1) for scoring
    abs( dwcagent_attr_output['namematch_distance'] - namematch_distance_max ) / namematch_distance_max * \
    (
        ( dwcagent_attr_output["custom_score_lifetime_data"] + dwcagent_attr_output['custom_score_multiple_names']) / 2
    )
).round(3)


dwcagent_attr_output['attributionRemarks'] = dwcagent_attr_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (data life time score);"
                " {lifetime_score_annote} (data life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_distance_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime_data"],
    lifetime_score_annote=row["custom_score_lifetime_data_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    by=['namematch_distance', 'family', 'given', 'custom_score_overall'], 
    ascending=[True, True, True, False], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
show_display_output=True
if show_display_output:
    display(dwcagent_attr_output.loc[pd.notnull(dwcagent_attr_output['yob'])].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob'].isnull()].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))


/tmp/ipykernel_41746/913966788.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_cossim.sort_values(


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,custom_score_lifetime_data,custom_score_lifetime_data_annotation
11330,A. Aaronsohn,Aaron Aaronsohn,0.50,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1876-1919,1.0,life time known
10333,S. Abedin,Sultanul Abedin,0.25,0.0 (k-means distance); 0.25 (score overall); ...,0.0,0.0,1952-?,0.5,year of death is missing
11331,A. Acebey,Amparo Acebey,0.25,0.0 (k-means distance); 0.25 (score overall); ...,0.0,0.0,1973-?,0.5,year of death is missing
2212,M. Ackermann,Markus Ackermann,0.00,0.0 (k-means distance); 0.00 (score overall); ...,-0.5,0.0,1967-?,0.5,year of death is missing
18057,J.G. Adam,Jacques-Georges Adam,0.50,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1909-1980,1.0,life time known


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,custom_score_lifetime_data,custom_score_lifetime_data_annotation
2213,M. Ackermann,Marianne Ackermann,-0.25,0.0 (k-means distance); -0.25 (score overall);...,-0.5,0.0,?-?,0.0,unknown life time
2214,M. Ackermann,Manfred Ackermann,-0.25,0.0 (k-means distance); -0.25 (score overall);...,-0.5,0.0,?-?,0.0,unknown life time
18549,L. Acosta,L. Acosta,0.00,0.0 (k-means distance); 0.00 (score overall); ...,0.0,0.0,?-?,0.0,unknown life time
21577,W. Adler,Wilhelm Adler,-0.25,0.0 (k-means distance); -0.25 (score overall);...,-0.5,0.0,?-?,0.0,unknown life time
18267,K. Adolphi,Klaus Adolphi,0.00,0.0 (k-means distance); 0.00 (score overall); ...,0.0,0.0,?-?,0.0,unknown life time


In [35]:
# refactor collectors_eventDate_mean
# refactor collectors_eventDate_min
# - refactor yob_is_lt_eventDate_min
# refactor collectors_eventDate_max
# - refactr yod_is_gt_eventDate_max
# refactor custom_score_lifetime            → custom_score_lifetime_data
# refactor custom_score_lifetime_annotation → custom_score_lifetime_data_annotation
# refactor namematch_similarity             → namematch_distance
# refactor namematch_similarity_annotation  → namematch_distance_annotation
# refactor custom_namematch_similarity      → custom_namematch_distance
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'namematch_distance':                 'custom_namematch_distance'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

show_display_output=False
if show_display_output:
    dwcagent_attr_output.head(20)

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 … represents the available data quality not the match in the first place
        'name',
        'attributionRemarks',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_distance',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime_data' # keep it for calculation convenience, no standard in DwC agent
    ]
)


In [36]:
show_display_output=True
if show_display_output:
    # criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda x: x < 0 ) # show multiple name cases
    
    display(dwcagent_attr_output[criterion].head(20))

,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,agentType,action,agentIdentifierType,identifier,custom_score_overall,custom_namematch_distance,custom_score_multiple_names,custom_score_lifetime_data
2212,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,1,Markus Ackermann,0.0 (k-means distance); 0.00 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504506,0.00,0.0,-0.5,0.5
2213,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,2,Marianne Ackermann,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q36619087,-0.25,0.0,-0.5,0.0
2214,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,3,Manfred Ackermann,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112660,-0.25,0.0,-0.5,0.0
21578,https://herbarium.bgbm.org/object/B101191388,"Adler,W.",W. Adler,1,Wolfgang Adler,0.0 (k-means distance); 0.00 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q119062956,0.00,0.0,-0.5,0.5
21577,https://herbarium.bgbm.org/object/B101191388,"Adler,W.",W. Adler,2,Wilhelm Adler,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q10393599,-0.25,0.0,-0.5,0.0
13971,https://herbarium.bgbm.org/object/B300230830,"Gradstein,S.R. & Aguirre,J.",J. Aguirre,1,Jaime Aguirre,0.0 (k-means distance); 0.00 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504658,0.00,0.0,-0.5,0.5
13972,https://herbarium.bgbm.org/object/B300230830,"Gradstein,S.R. & Aguirre,J.",J. Aguirre,2,Julio Aguirre,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112740,-0.25,0.0,-0.5,0.0
20800,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,1,Sultan Ahmad,0.0 (k-means distance); 0.25 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112765,0.25,0.0,-0.5,1.0
20799,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,2,Shamsuddin Ahmad,0.0 (k-means distance); 0.00 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q33661104,0.00,0.0,-0.5,0.5
20801,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,3,Suhail Ahmad,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q88848655,-0.25,0.0,-0.5,0.0


In [37]:
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
this_timestamp_for_data=20231116
this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_dwc-agent-output_20231116.csv (7896 kB)


## Documentation

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
namematch_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.) the first date of eventDate (supposedly the first sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.) the last date of eventDate (supposedly the last sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))

Refactoring from <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>

AVH | collector_matching (here)
-|-
avh_matches | collectors_all_matches
wd_test | wd_matchtest